In [1]:
from FinMind.data import DataLoader
from dotenv import load_dotenv
import os

load_dotenv()
token = os.getenv("API_TOKEN")

api = DataLoader()
api.login_by_token(api_token=token)

In [2]:
start_date = "1994-10-02"
days_ago = 20
stock_id = "2385"

candle_low = 0.03
volume_x = 3

tp = 0.05
sl = -0.03

In [3]:
taiwan_stocks = api.taiwan_stock_info()

# Extract stock ids
stock_ids = taiwan_stocks['stock_id'].tolist()

print(f"Total stocks fetched: {len(stock_ids)}")

2024-03-28 21:59:05.340 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockInfo, data_id: 


Total stocks fetched: 3241


In [4]:
stock_ids

['0050',
 '0051',
 '0052',
 '0053',
 '0054',
 '0055',
 '0056',
 '0057',
 '0058',
 '0059',
 '0060',
 '0061',
 '006201',
 '006203',
 '006204',
 '006205',
 '006206',
 '006207',
 '006208',
 '00625K',
 '00631L',
 '00632R',
 '00633L',
 '00634R',
 '00635U',
 '00636',
 '00636K',
 '00637L',
 '00638R',
 '00639',
 '00640L',
 '00641R',
 '00642U',
 '00643',
 '00643K',
 '00645',
 '00646',
 '00647L',
 '00648R',
 '00649',
 '00650L',
 '00651R',
 '00652',
 '00653L',
 '00654R',
 '00655L',
 '00656R',
 '00657',
 '00657K',
 '00658L',
 '00659R',
 '00660',
 '00661',
 '00662',
 '00663L',
 '00664R',
 '00665L',
 '00666R',
 '00667',
 '00668',
 '00668K',
 '00669R',
 '00670L',
 '00671R',
 '00672L',
 '00673R',
 '00674R',
 '00675L',
 '00676R',
 '00677U',
 '00678',
 '00679B',
 '00680L',
 '00681R',
 '00682U',
 '00683L',
 '00684R',
 '00685L',
 '00686R',
 '00687B',
 '00688L',
 '00689R',
 '00690',
 '00691R',
 '00692',
 '00693U',
 '00694B',
 '00695B',
 '00696B',
 '00697B',
 '00698L',
 '00699R',
 '00700',
 '00701',
 '00702'

In [5]:
import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [6]:
# Get the current time as a datetime object
current_datetime = datetime.now()

# Calculate the datetime for 30 days ago
n_days_ago_datetime = current_datetime - timedelta(days=days_ago)

# Format both dates as 'YYYY-MM-DD'
current_date_str = current_datetime.strftime('%Y-%m-%d')
n_days_ago_date_str = n_days_ago_datetime.strftime('%Y-%m-%d')

print(f"Current Date: {current_date_str}")
print(f"Date {days_ago} Days Ago: {n_days_ago_date_str}")

Current Date: 2024-03-28
Date 20 Days Ago: 2024-03-08


In [7]:
df = api.taiwan_stock_daily_adj(
    stock_id = stock_id,
    start_date = start_date,
    end_date = current_date_str,
)

2024-03-28 21:59:11.557 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPriceAdj, data_id: 2385


In [8]:
df

,date,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover
0,1999-01-06,2385,3216091,147205926,6.76,6.98,6.76,6.82,-0.44,1126
1,1999-01-07,2385,2645941,116716265,6.82,6.83,6.34,6.83,0.01,1006
2,1999-01-08,2385,2096589,96009854,6.61,6.88,6.61,6.85,0.02,770
3,1999-01-11,2385,1329400,59643244,6.83,6.83,6.47,6.70,-0.15,484
4,1999-01-12,2385,1799002,78059456,6.53,6.58,6.31,6.31,-0.39,429
...,...,...,...,...,...,...,...,...,...,...
6088,2024-03-22,2385,15597299,3691300241,244.00,247.50,230.50,232.50,-14.50,13658
6089,2024-03-25,2385,9687048,2204182227,231.50,237.00,223.00,223.00,-9.50,13781
6090,2024-03-26,2385,13873390,3000683326,223.00,225.50,209.00,213.50,-9.50,13678
6091,2024-03-27,2385,15252166,3316574852,216.00,221.50,213.00,216.00,2.50,15013


In [9]:
def trades(df):
    # condition 1
    c1_green = ((df['open'] - df['min']) / df['open']) >= candle_low
    c1_red = ((df['close'] - df['min']) / df['open']) >= candle_low
    c1 = ((df['close'] > df['open']) & c1_green) | ((df['open'] > df['close']) & c1_red)
    
    # condition 2
    rolling_avg_volume = df['Trading_Volume'].rolling(window=days_ago).mean()
    c2 = df['Trading_Volume'] >= rolling_avg_volume * volume_x
    
    # Combine conditions
    conditions_met = c1 & c2
    
    conditions_met_shifted = conditions_met.shift(1, fill_value=False)
    
    df['trade_entry'] = conditions_met_shifted
    df['exit_signal'] = False
    df['next_day_open'] = df['open'].shift(-1)
    
    # Iterate through DataFrame to simulate trade entries and exits
    in_trade = False
    entry_price = None
    
    for i in range(len(df)):
        if not in_trade:
            # Check if we should enter a trade
            if df.iloc[i]['trade_entry']:
                in_trade = True
                entry_price = df.iloc[i]['open']
                df.at[i, 'entry_price'] = entry_price
        else:
            # Calculate price changes since entry
            max_pct_change = (df.iloc[i]['max'] - entry_price) / entry_price
            min_pct_change = (df.iloc[i]['min'] - entry_price) / entry_price
            
            exit_based_on_close = (df.iloc[i]['close'] - entry_price) / entry_price
            
            # Check exit criteria: max reaching 5% or min reaching -3%
            if max_pct_change >= tp or min_pct_change <= sl:
                df.at[i, 'exit_signal'] = True
                df.at[i, 'exit_price'] = entry_price * (1 + tp if max_pct_change >= tp else 1 + sl)
                in_trade = False
                entry_price = None
    
    # Filter to find rows where a trade was exited
    # trades_exited = df[df['exit_signal'] | df['trade_entry']]
    
    # print(trades_exited[['open', 'max', 'min', 'close', 'entry_price', 'exit_price', 'trade_entry', 'exit_signal']])

In [10]:
# df.to_csv("data.csv")

In [11]:
# Initialize variables for calculating wins and total profits
def win_profit(df):    
    total_trades = 0
    wins = 0
    total_profit = 0.0
    entry_size_ntd = 1000000
    
    for i in df.index[df['exit_signal']]:
        # Create a boolean array up to the current index i, indicating where trade_entry is True
        trade_entries_up_to_i = df['trade_entry'][:i+1]
        if trade_entries_up_to_i.any():
            # Find the last True 'trade_entry' index before the current 'exit_signal'
            entry_index = trade_entries_up_to_i[trade_entries_up_to_i].index[-1]
            entry_price = df.loc[entry_index, 'entry_price']
            exit_price = df.loc[i, 'exit_price']
            shares_bought = entry_size_ntd / (entry_price * 1000)
            profit = (exit_price - entry_price) * shares_bought * 1000
            total_profit += profit
            total_trades += 1
            if profit > 0:
                wins += 1
    
    # Avoid division by zero
    win_rate = (wins / total_trades) * 100 if total_trades > 0 else 0

    return win_rate, total_profit

# print(f"Win Rate: {win_rate:.2f}%")
# print(f"Total Profit: {total_profit}")

In [12]:
accumulated_data = []

In [13]:
for stock_id in stock_ids:
    try:
        df = api.taiwan_stock_daily_adj(
            stock_id=stock_id,
            start_date=start_date,
            end_date=current_date_str,
        )
        
        trades(df)
        rate, profit = win_profit(df)
        print(rate, profit)
        
        # Append the results for the current stock to the consolidated DataFrame
        accumulated_data.append({
            'stock_id': stock_id,
            'win_rate': rate,
            'total_profit': profit
        })
        
    except Exception as e:
        print(f"Error processing {stock_id}: {e}")

# Now, consolidated_results contains the win rate and total profit for each stock that was processed
consolidated_results = pd.DataFrame(accumulated_data)
print(consolidated_results)

2024-03-28 21:59:14.566 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPriceAdj, data_id: 0050
2024-03-28 21:59:17.781 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPriceAdj, data_id: 0051


80.0 170000.00000000012


2024-03-28 21:59:21.227 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPriceAdj, data_id: 0052


33.33333333333333 -10000.000000000156


2024-03-28 21:59:24.099 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPriceAdj, data_id: 0053


...

2024-03-28 23:01:48.819 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPriceAdj, data_id: 3202


50.0 199999.99999999997


2024-03-28 23:01:51.418 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPriceAdj, data_id: 3205


36.84210526315789 nan



KeyboardInterrupt



In [14]:
accumulated_data

[{'stock_id': '0050', 'win_rate': 80.0, 'total_profit': 170000.00000000012},
 {'stock_id': '0051',
  'win_rate': 33.33333333333333,
  'total_profit': -10000.000000000156},
 {'stock_id': '0052', 'win_rate': 0, 'total_profit': 0.0},
 {'stock_id': '0053', 'win_rate': 50.0, 'total_profit': 20000.000000000033},
 {'stock_id': '0054', 'win_rate': 50.0, 'total_profit': 39999.999999999956},
 {'stock_id': '0055', 'win_rate': 100.0, 'total_profit': 50000.00000000012},
 {'stock_id': '0056', 'win_rate': 100.0, 'total_profit': 100000.00000000015},
 {'stock_id': '0057', 'win_rate': 100.0, 'total_profit': 49999.99999999996},
 {'stock_id': '0058',
  'win_rate': 33.33333333333333,
  'total_profit': -10000.000000000044},
 {'stock_id': '0059',
  'win_rate': 33.33333333333333,
  'total_profit': -9999.999999999982},
 {'stock_id': '0060', 'win_rate': 50.0, 'total_profit': 40000.00000000006},
 {'stock_id': '0061', 'win_rate': 0, 'total_profit': 0.0},
 {'stock_id': '006201', 'win_rate': 100.0, 'total_profit': 

In [19]:
consolidated_results = pd.DataFrame(accumulated_data)
print(consolidated_results)
consolidated_results.to_csv("backtest.csv")

     stock_id   win_rate  total_profit
0        0050  80.000000      170000.0
1        0051  33.333333      -10000.0
2        0052   0.000000           0.0
3        0053  50.000000       20000.0
4        0054  50.000000       40000.0
...       ...        ...           ...
1445     3188  33.333333           NaN
1446     3189  40.000000       10000.0
1447     3189  40.000000       10000.0
1448     3191  50.000000      200000.0
1449     3202  36.842105           NaN

[1450 rows x 3 columns]


In [17]:
df

,date,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover,trade_entry,exit_signal,next_day_open,entry_price,exit_price
0,2005-07-27,3202,3,65,15.37,15.37,15.37,15.37,0.99,3,False,False,16.43,NaN,NaN
1,2005-07-28,3202,1,23,16.43,16.43,16.43,16.43,1.06,1,False,False,17.56,NaN,NaN
2,2005-07-29,3202,1,25,17.56,17.56,17.56,17.56,1.13,1,False,False,18.77,NaN,NaN
3,2005-08-01,3202,1,27,18.77,18.77,18.77,18.77,1.21,1,False,False,20.08,NaN,NaN
4,2005-08-02,3202,1,28,20.08,20.08,20.08,20.08,1.31,1,False,False,21.46,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4594,2024-03-22,3202,844000,10496650,12.50,12.65,12.25,12.30,0.00,288,False,False,12.30,NaN,NaN
4595,2024-03-25,3202,978000,12147500,12.30,12.65,12.15,12.60,0.30,260,False,False,12.60,NaN,NaN
4596,2024-03-26,3202,218000,2728500,12.60,12.70,12.35,12.45,-0.15,124,False,False,12.30,NaN,NaN
4597,2024-03-27,3202,394000,4848450,12.30,12.45,12.20,12.40,-0.05,169,False,False,12.30,NaN,NaN
